In [87]:
import pandas as pd  
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.dates as mdates

In [105]:
# Load the CSV Data
df= pd.read_csv('../data/handlebar/Accelerometer_data_merged_ESP1.csv')  # Load the CSV data using pandas
df['NTP'] = pd.to_datetime(df['NTP'])
# Filter the DataFrame to include only the necessary columns
df_filtered = df[['NTP','Acc-Z','curb_scene']]
df_filtered.head()

,NTP,Acc-Z,curb_scene
0,2024-07-24 10:26:01.863,8.762772,0.0
1,2024-07-24 10:26:01.867,8.753204,0.0
2,2024-07-24 10:26:01.872,8.762772,0.0
3,2024-07-24 10:26:01.877,8.781937,0.0
4,2024-07-24 10:26:01.884,8.753204,0.0


In [106]:
def handle_missing_values_length(df_filtered):
    # Handle missing values length
    missing_info = {}  # List to store (start_index, current_count)
    current_count = 0
    start_index = None  # To store the start time of missing values
    for index, row in df_filtered.iterrows():
        if np.isnan(row['Acc-Z']):
            if start_index is None:
                start_index = index
                current_count += 1
            if start_index is not None:
                current_count += 1
        # If the value is not NaN, continue sequence broke
        else:
            if start_index is not None:
                missing_info[start_index] = current_count
                start_index = None
                current_count = 0

    # Handle case where the last segment has missing values
    if current_count > 0:
        missing_info[start_index] = current_count

    for key, value in missing_info.items():
        print(key, value)
        if value > 100:
            print("Start Index:", key, "Count:", value)  
    print("No missing value longer than half a second")

    return missing_info


In [109]:
# Function to fill missing values based on the specified conditions
def fill_missing_values(df):
    for index in range(1, len(df)):
        if pd.isnull(df['Acc-Z'].iloc[index]):
            if df['curb_scene'].iloc[index - 1] == df['curb_scene'].iloc[index]:
                df.at[index, 'Acc-Z'] = df['Acc-Z'].iloc[index - 1]
                df.at[index, 'NTP'] = df['NTP'].iloc[index - 1] + pd.Timedelta(milliseconds=1)
            else:
                # Find the next non-NaN value below
                for j in range(index + 1, len(df)):
                    if not pd.isnull(df['Acc-Z'].iloc[j]):
                        df.at[index, 'Acc-Z'] = df['Acc-Z'].iloc[j]
                        df.at[index, 'NTP'] = df['NTP'].iloc[j] - pd.Timedelta(milliseconds=1)
                        break
    return df

In [ ]:
#check for missing values longer than 0.5 seconds
#missing_info = handle_missing_values_length(df_filtered)

In [ ]:
#fill missing values
fill_missing_values(df_filtered)

,NTP,Acc-Z,curb_scene
0,2024-07-24 10:26:01.863,8.762772,0.0
1,2024-07-24 10:26:01.867,8.753204,0.0
2,2024-07-24 10:26:01.872,8.762772,0.0
3,2024-07-24 10:26:01.877,8.781937,0.0
4,2024-07-24 10:26:01.884,8.753204,0.0
...,...,...,...
549230,2024-07-24 11:11:20.851,8.777145,0.0
549231,2024-07-24 11:11:20.857,8.681381,0.0
549232,2024-07-24 11:11:20.866,8.370132,0.0
549233,2024-07-24 11:11:20.870,8.307877,0.0


In [ ]:
# Check if there are any missing values
#missing_info= handle_missing_values_length(df_filtered)

No missing value longer than half a second


In [114]:
# #check some random numbers:
# print(df_filtered.loc[21827 -10:21827 +5])
# print(df_filtered.loc[41417 -10:41417 +5]) 
# print(df_filtered.loc[41458 -10:41458 +5])


In [ ]:
# slicing the data to get the curb scene in 0.5 seconds
# 1. check if they have the same curb scene
# 2. I want for each curb scene to have 100 samples
# 3. if the last curb scene is too short, i want to what before until it reaches 100 samples
